In [35]:
from datetime import datetime
from utils import utils


start_timestamp = utils.date_to_utc_timestamp(datetime(2023, 5, 23, 0, 0))
end_timestamp = utils.date_to_utc_timestamp(datetime(2023, 5, 26, 0, 0))

In [36]:
# save kucoin price weekly data
import requests


kucoin_url = "https://min-api.cryptocompare.com/data/v2/histominute?fsym=ETH&tsym=USD&limit=1000&toTs={}"

current_timestamp = end_timestamp
# current_timestamp = utils.date_to_utc_timestamp(datetime(2023, 5, 19, 1, 32))
kucoin_result = []
while current_timestamp >= start_timestamp:
    print("Start processing, current time: ", utils.utc_timestamp_to_date(current_timestamp))
    response = requests.get(kucoin_url.format(current_timestamp))
    kucoin_data = response.json()['Data']
    if not kucoin_data:
        break
    for d in kucoin_data['Data'][::-1]:
        if d["time"] >= start_timestamp:
            kucoin_result.append(d)
        else:
            break
    current_timestamp = kucoin_data['TimeFrom'] - 60

kucoin_result = kucoin_result[::-1]

print(len(kucoin_result))

Start processing, current time:  2023-05-26 00:00:00+00:00
Start processing, current time:  2023-05-25 07:19:00+00:00
Start processing, current time:  2023-05-24 14:38:00+00:00
Start processing, current time:  2023-05-23 21:57:00+00:00
Start processing, current time:  2023-05-23 05:16:00+00:00
4321


In [19]:
print(kucoin_result[0])

{'time': 1684460940, 'high': 1798.38, 'low': 1797.75, 'open': 1797.75, 'volumefrom': 57.77, 'volumeto': 103880.66, 'close': 1798.05, 'conversionType': 'direct', 'conversionSymbol': ''}


In [22]:
# ARB graphQL
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport



arb_query = """
query {
    swaps(skip: %d, orderBy: timestamp, orderDirection: asc,
        where: { pool: "0xc31e54c7a869b9fcbecc14363cf510d1c41fa443", timestamp_gte: %d }){
            id
            amountIn
            amountInUSD
            amountOut
            amountOutUSD
            tick
            timestamp
            tokenIn {
                symbol
                decimals
            }
            tokenOut {
                symbol
                decimals
            }
    }
}
"""


client = Client(
    transport=RequestsHTTPTransport(
        url='https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-arbitrum',
        verify=True,
        retries=5,
    ))

# # variables = { "num_skip": 0, "start_timestamp": 1684460940 }
# # response = client.execute(gql(arb_query), variable_values=variables)
response = client.execute(gql(arb_query % (0, start_timestamp)))
print(len(response["swaps"]))

100


In [23]:
print(response["swaps"][0])

# temp
result = []
id_set = set()

{'id': '0x13875c5e00916a733dbf870b03f29c7c0e2b575294ea3c17e3a123540b281eca0b000000', 'amountIn': '18509968', 'amountInUSD': '18.509968', 'amountOut': '10174988585088354', 'amountOutUSD': '18.50071257554249565419706', 'tick': '-201264', 'timestamp': '1684800030', 'tokenIn': {'symbol': 'USDC', 'decimals': 6}, 'tokenOut': {'symbol': 'WETH', 'decimals': 18}}


In [50]:
# response = client.execute(gql(arb_query % start_timestamp))
# result = response['swaps']
# id_set = set()
#
# for item in result:
#     id_set.add(item["id"])

skip = 0
last_cnt = 0
while int(result[-1]["timestamp"]) <= end_timestamp:
    print(datetime.now(), "result cnt:", len(result), "id cnt:", len(id_set), "skip: ", skip)
    print("Current time: ", utils.utc_timestamp_to_date(int(result[-1]["timestamp"])))
    last_cnt = len(result)
    current_timestamp = int(result[-1]["timestamp"])
    query = arb_query % (skip, current_timestamp)
    try:
        response = client.execute(gql(query))

        swap_data = response["swaps"]
        if len(swap_data) == 0:
            break
        for item in swap_data:
            if int(item["timestamp"]) > end_timestamp:
                break
            elif item["id"] in id_set:
                continue
            else:
                result.append(item)
                id_set.add(item["id"])

        if last_cnt == len(result):
            skip += 100
        else:
            skip = 0
    except Exception as ex:
        print("Fetching swap data error", ex)

2023-05-26 19:06:51.391025 result cnt: 23386 id cnt: 23386 skip:  0
Current time:  2023-05-24 13:52:26+00:00
2023-05-26 19:06:56.193864 result cnt: 23386 id cnt: 23386 skip:  100
Current time:  2023-05-24 13:52:26+00:00
2023-05-26 19:07:00.436694 result cnt: 23486 id cnt: 23486 skip:  0
Current time:  2023-05-24 13:55:08+00:00
2023-05-26 19:07:04.621134 result cnt: 23585 id cnt: 23585 skip:  0
Current time:  2023-05-24 13:58:02+00:00
2023-05-26 19:07:08.786921 result cnt: 23684 id cnt: 23684 skip:  0
Current time:  2023-05-24 14:02:25+00:00
2023-05-26 19:07:13.029153 result cnt: 23783 id cnt: 23783 skip:  0
Current time:  2023-05-24 14:06:10+00:00
2023-05-26 19:07:17.294734 result cnt: 23879 id cnt: 23879 skip:  0
Current time:  2023-05-24 14:06:43+00:00
2023-05-26 19:07:21.578973 result cnt: 23977 id cnt: 23977 skip:  0
Current time:  2023-05-24 14:08:55+00:00
2023-05-26 19:07:25.795697 result cnt: 24076 id cnt: 24076 skip:  0
Current time:  2023-05-24 14:13:49+00:00
2023-05-26 19:07:

KeyboardInterrupt: 

In [41]:
query = arb_query % current_timestamp
response = client.execute(gql(query))
swap_data = response["swaps"]
print(len(swap_data))
print(len(id_set))

100
23386


In [45]:
print(current_timestamp)
print(item["timestamp"])

1684936346
1684936346


In [46]:
print(swap_data)

[{'id': '0x12a70eac042cf435da93d9cdca04f64d4abd946270608e8f6738536fb116cba115000000', 'amountIn': '631949258', 'amountInUSD': '631.949258', 'amountOut': '351717616321909247', 'amountOutUSD': '631.8643907572813422825935', 'tick': '-201388', 'timestamp': '1684936346', 'tokenIn': {'symbol': 'USDC', 'decimals': 6}, 'tokenOut': {'symbol': 'WETH', 'decimals': 18}}, {'id': '0x71a4802f92e6d6d8f283daaaa73cf66e3bb0a87b5c58ea9ee3a2b00b237d507301020000', 'amountIn': '88000000', 'amountInUSD': '88', 'amountOut': '48975992489322807', 'amountOutUSD': '87.9858847549895606649735', 'tick': '-201388', 'timestamp': '1684936346', 'tokenIn': {'symbol': 'USDC', 'decimals': 6}, 'tokenOut': {'symbol': 'WETH', 'decimals': 18}}, {'id': '0x71a4802f92e6d6d8f283daaaa73cf66e3bb0a87b5c58ea9ee3a2b00b237d507304030000', 'amountIn': '88000000', 'amountInUSD': '88', 'amountOut': '48975345149648370', 'amountOutUSD': '87.984721802467368012885', 'tick': '-201388', 'timestamp': '1684936346', 'tokenIn': {'symbol': 'USDC', 'dec

In [44]:
time_cnt = 0
id_cnt = 0
for item in swap_data:
    if int(item["timestamp"]) > current_timestamp:
        time_cnt += 1
    elif item["id"] in id_set:
        id_cnt += 1

print(time_cnt)
print(id_cnt)

0
100


In [37]:
# save data
import pandas as pd

kucoin_df = pd.DataFrame(kucoin_result)
kucoin_df.to_csv("../../data/kucoin_eth_price.csv", index=False)

In [51]:
import pandas as pd
from copy import deepcopy


# arb_result = deepcopy(result)
arb_result1 = deepcopy(result)
for item in arb_result1:
    item["tokenIn"] = item["tokenIn"]["symbol"]
    item["tokenOut"] = item["tokenOut"]["symbol"]

arb_uniswap_df = pd.DataFrame(arb_result1)
arb_uniswap_df.to_csv("../../data/arb_swap1.csv", index=False)

In [29]:
import math

def sqrtpricex96_to_price_eth_usd(sqrtpricex96):
    """Converts a Uniswap SQRTpriceX96 to price.

    Args:
      sqrtpricex96: The SQRTpriceX96 to convert.

    Returns:
      The price.
    """
    price = sqrtpricex96 / math.pow(2, 96)
    return 1 / (price ** 2 / (10 ** 12))


print(sqrtpricex96_to_price_eth_usd(1874659030893028805438655155008306))

1786.1362752201069


In [31]:
TICK_BASE = 1.0001


def tick_to_price_eth_usd(tick):
    return 1 / (TICK_BASE ** tick / (10 ** 12))


print(tick_to_price_eth_usd(201442))

1786.1687802452334


In [36]:
def arb_tick_to_price_eth_usd(tick):
    return TICK_BASE ** tick * 10 ** 12

print(arb_tick_to_price_eth_usd(-201439))

1786.7046844661563


In [1]:
# read kucoin & arb data
import pandas as pd

kucoin_df = pd.read_csv("../../data/kucoin_eth_price.csv")
arb_df = pd.read_csv("../../data/arb_swap_all.csv")

In [2]:
arb_df.head()

,id,amountIn,amountInUSD,amountOut,amountOutUSD,tick,timestamp,tokenIn,tokenOut
0,0x13875c5e00916a733dbf870b03f29c7c0e2b575294ea...,18509968,18.509968,10174988585088354,18.500713,-201264,1684800030,USDC,WETH
1,0xa629bdac51ec7b793a04193f87006f0ff4ae910f8025...,417246407,417.246407,229361579040734245,417.037599,-201264,1684800044,USDC,WETH
2,0x38d8de741acaf8f47d3e1692926b7a663e93b6fc0e4b...,15034268999359802,27.336143,27322474,27.322474,-201264,1684800056,WETH,USDC
3,0x452dd1fbd5a7c6d03880cd92be8bc35646422b627c34...,40248724,40.248724,22124832513497315,40.228597,-201264,1684800071,USDC,WETH
4,0x7ffaad3f6e94d4e7d19df8b5f8c808512918acccf5f1...,54000000000000000,98.185802,98136699,98.136699,-201264,1684800077,WETH,USDC


In [3]:
kucoin_df.head()

,time,high,low,open,volumefrom,volumeto,close,conversionType,conversionSymbol
0,1684800000,1818.27,1817.80,1817.80,54.66,99368.32,1818.17,direct,NaN
1,1684800060,1818.40,1818.17,1818.17,38.42,69844.82,1818.38,direct,NaN
2,1684800120,1818.48,1818.23,1818.38,30.73,55879.84,1818.41,direct,NaN
3,1684800180,1818.99,1818.41,1818.41,101.61,184832.72,1818.85,direct,NaN
4,1684800240,1818.93,1818.52,1818.85,47.14,85728.63,1818.85,direct,NaN


In [6]:
# convert arb swap data to minute price
def get_trade_data(swap_dict):
    if swap_dict["tokenIn"] == "WETH":
        eth_amount = int(swap_dict["amountIn"]) / (10 ** 18)
        usd = float(swap_dict["amountInUSD"])
    else:
        eth_amount = int(swap_dict["amountOut"]) / (10 ** 18)
        usd = float(swap_dict["amountOutUSD"])

    if eth_amount != 0:
        return usd / eth_amount, usd
    return 0, usd


arb_result = arb_df.to_dict("records")

error_result = []
new_result = []
for r in arb_result:
    price, trade_usd = get_trade_data(r)
    if price == 0:
        error_result.append(r)
    else:
        r["price"] = price
        r["trade_usd"] = trade_usd
        new_result.append(r)


print(len(error_result))
print(len(new_result))
print(new_result[0])

2
46367


In [8]:
kucoin_time_list = kucoin_df["time"].tolist()[::-1]

def get_time_key(timestamp):
    end_time = kucoin_time_list[0] + 60

    for t in kucoin_time_list:
        if t < timestamp <= end_time:
            return t
    return None


minute_price_dict = dict()

for r in new_result[::-1]:
    swap_time = r["timestamp"]
    time_key = get_time_key(swap_time)
    if time_key not in minute_price_dict:
        minute_price_dict[time_key] = [(r["price"], r["trade_usd"])]
    elif time_key is not None:
        minute_price_dict[time_key].append((r["price"], r["trade_usd"]))


# stats minute price
print(len(new_result))
print(len(minute_price_dict.keys()))
print(list(minute_price_dict.keys())[:10])

46367
4272
[1685059080, 1685059020, 1685058960, 1685058900, 1685058840, 1685058780, 1685058720, 1685058660, 1685058540, 1685058480]


In [9]:
import pandas as pd


def get_price_detail(trade_list):
    price_detail_dict = dict()
    price_list = [p[0] for p in trade_list]
    price_detail_dict["open"] = price_list[-1]
    price_detail_dict["close"] = price_list[0]
    price_detail_dict["high"] = max(price_list)
    price_detail_dict["low"] = min(price_list)

    usd_list = [p[1] for p in trade_list]
    price_detail_dict["volume"] = sum(usd_list)
    return price_detail_dict


minute_stats = []
for time_key in sorted(minute_price_dict.keys()):
    stats_dict = get_price_detail(minute_price_dict[time_key])
    stats_dict["time"] = time_key
    minute_stats.append(stats_dict)


arb_df = pd.DataFrame(minute_stats)
arb_df.head()

,open,close,high,low,volume,time
0,1818.25389,1818.25555,1818.25555,1818.25389,462.874455,1684800000
1,1818.25545,1818.25784,1818.25784,1818.25545,2996.957906,1684800060
2,1818.24957,1818.37415,1818.37415,1818.24957,32740.669877,1684800180
3,1818.37420,1818.37420,1818.37420,1818.37420,55.016836,1684800240
4,1818.37399,1818.37403,1818.37403,1818.37399,32740.751811,1684800300


In [11]:
# arb_df join kucoin_df by time schema and left high, low schema
df = pd.merge(arb_df, kucoin_df, on='time')[["time", "high_x", "low_x", "high_y", "low_y", "volume", "volumeto"]]
df.head()

,time,high_x,low_x,high_y,low_y,volume,volumeto
0,1684800000,1818.25555,1818.25389,1818.27,1817.80,462.874455,99368.32
1,1684800060,1818.25784,1818.25545,1818.40,1818.17,2996.957906,69844.82
2,1684800180,1818.37415,1818.24957,1818.99,1818.41,32740.669877,184832.72
3,1684800240,1818.37420,1818.37420,1818.93,1818.52,55.016836,85728.63
4,1684800300,1818.37403,1818.37399,1819.17,1818.85,32740.751811,49105.83


In [12]:
df.rename(columns={
    "high_x": "arb_high",
    "low_x": "arb_low",
    "high_y": "kucoin_high",
    "low_y": "kucoin_low",
    "volume": "arb_volume",
    "volumeto": "kucoin_volume",
}, inplace=True)

df.head()

,time,arb_high,arb_low,kucoin_high,kucoin_low,arb_volume,kucoin_volume
0,1684800000,1818.25555,1818.25389,1818.27,1817.80,462.874455,99368.32
1,1684800060,1818.25784,1818.25545,1818.40,1818.17,2996.957906,69844.82
2,1684800180,1818.37415,1818.24957,1818.99,1818.41,32740.669877,184832.72
3,1684800240,1818.37420,1818.37420,1818.93,1818.52,55.016836,85728.63
4,1684800300,1818.37403,1818.37399,1819.17,1818.85,32740.751811,49105.83


In [13]:
df["high_diff"] = df["arb_high"] - df["kucoin_high"]
df["high_diff_rate"] = abs(df["high_diff"]) / df["arb_high"]
df["low_diff"] = df["arb_low"] - df["kucoin_low"]
df["low_diff_rate"] = abs(df["low_diff"]) / df["arb_low"]

df.head()

,time,arb_high,arb_low,kucoin_high,kucoin_low,arb_volume,kucoin_volume,high_diff,high_diff_rate,low_diff,low_diff_rate
0,1684800000,1818.25555,1818.25389,1818.27,1817.80,462.874455,99368.32,-0.01445,0.000008,0.45389,0.000250
1,1684800060,1818.25784,1818.25545,1818.40,1818.17,2996.957906,69844.82,-0.14216,0.000078,0.08545,0.000047
2,1684800180,1818.37415,1818.24957,1818.99,1818.41,32740.669877,184832.72,-0.61585,0.000339,-0.16043,0.000088
3,1684800240,1818.37420,1818.37420,1818.93,1818.52,55.016836,85728.63,-0.55580,0.000306,-0.14580,0.000080
4,1684800300,1818.37403,1818.37399,1819.17,1818.85,32740.751811,49105.83,-0.79597,0.000438,-0.47601,0.000262


In [15]:
df.sort_values(by=["high_diff_rate"], ascending=False, inplace=True)
df.head(20)

,time,arb_high,arb_low,kucoin_high,kucoin_low,arb_volume,kucoin_volume,high_diff,high_diff_rate,low_diff,low_diff_rate
195,1684812000,1867.37294,1852.71090,1870.55,1853.98,3.894675e+06,9282686.39,-3.17706,0.001701,-1.26910,0.000685
2931,1684978020,1770.01292,1767.47280,1767.34,1764.28,4.514917e+05,2724207.96,2.67292,0.001510,3.19280,0.001806
1629,1684899360,1825.65451,1820.21040,1823.10,1813.17,1.119969e+06,5077359.02,2.55451,0.001399,7.04040,0.003868
890,1684853760,1853.62754,1853.61276,1856.00,1852.99,2.993200e+04,4139321.74,-2.37246,0.001280,0.62276,0.000336
894,1684854000,1856.54054,1856.54027,1858.85,1856.94,2.664802e+02,1634661.79,-2.30946,0.001244,-0.39973,0.000215
2932,1684978080,1769.04036,1767.46773,1766.89,1763.39,4.842366e+05,4177774.37,2.15036,0.001216,4.07773,0.002307
1635,1684899720,1820.02112,1819.42965,1817.87,1816.45,1.223121e+05,1008461.31,2.15112,0.001182,2.97965,0.001638
1634,1684899660,1820.28375,1820.05236,1818.35,1816.93,5.318695e+04,599747.13,1.93375,0.001062,3.12236,0.001716
1701,1684903680,1826.06242,1826.06147,1828.00,1826.93,2.783265e+02,780518.72,-1.93758,0.001061,-0.86853,0.000476
3413,1685007060,1787.06384,1786.67272,1788.96,1786.94,7.781383e+04,2127254.84,-1.89616,0.001061,-0.26728,0.000150


In [16]:
df.sort_values(by=["low_diff_rate"], ascending=False, inplace=True)
df.head(20)

,time,arb_high,arb_low,kucoin_high,kucoin_low,arb_volume,kucoin_volume,high_diff,high_diff_rate,low_diff,low_diff_rate
3531,1685014140,1786.48266,1700.00000,1785.99,1784.80,5.971460e+05,1254389.71,0.49266,0.000276,-84.80000,0.049882
2714,1684964760,1801.35433,1750.00000,1800.90,1794.74,1.287568e+06,2165398.44,0.45433,0.000252,-44.74000,0.025566
2930,1684977960,1781.39048,1769.13005,1779.60,1761.75,1.738033e+06,12016874.02,1.79048,0.001005,7.38005,0.004172
2259,1684937160,1794.26047,1785.04584,1793.17,1777.98,1.326263e+06,8931168.39,1.09047,0.000608,7.06584,0.003958
1629,1684899360,1825.65451,1820.21040,1823.10,1813.17,1.119969e+06,5077359.02,2.55451,0.001399,7.04040,0.003868
2245,1684936320,1804.98630,1795.92821,1803.49,1790.39,2.635538e+06,6926729.09,1.49630,0.000829,5.53821,0.003084
2932,1684978080,1769.04036,1767.46773,1766.89,1763.39,4.842366e+05,4177774.37,2.15036,0.001216,4.07773,0.002307
1630,1684899420,1821.08974,1820.39379,1820.27,1816.49,1.620217e+05,1437353.56,0.81974,0.000450,3.90379,0.002144
1628,1684899300,1828.99055,1825.65451,1828.91,1821.92,8.734903e+05,2915011.75,0.08055,0.000044,3.73451,0.002046
1631,1684899480,1821.00177,1820.73947,1819.30,1817.37,7.217439e+04,502431.15,1.70177,0.000935,3.36947,0.001851


In [19]:
minute_price_dict[1684977960]

[(1769.4293599999999, 6626.393923568697),
 (1769.4293599999999, 250.3699255439876),
 (1769.13005, 31844.3409),
 (1769.13005, 100.94920742282656),
 (1769.13005, 100.94926054685985),
 (1769.1300499999998, 100.94931367093506),
 (1769.13005, 100.94936679505221),
 (1769.1300499999998, 100.9494199192113),
 (1769.13005, 100.94947304341233),
 (1769.6466400000002, 56046.832664768),
 (1769.6527699999997, 664.4461800418076),
 (1769.65071, 223.4340346378958),
 (1769.73482, 1859.2483910182832),
 (1769.7348200000001, 4067.9090656855537),
 (1769.7348200000001, 6916.182520152976),
 (1769.80622, 227.570417136639),
 (1769.8062200000002, 7972.463981541213),
 (1770.84708, 5699.327669312736),
 (1770.84708, 124.198574499944),
 (1770.8470799999998, 262.81950651685935),
 (1770.8470800000002, 166.0443052948351),
 (1770.84708, 716.0580468532855),
 (1770.8470799999998, 1280.009460957504),
 (1770.84708, 70.00509938215207),
 (1770.8470799999998, 107.75681982348422),
 (1770.8470799999998, 132.3170718961055),
 (1770

In [26]:
# price fast change period
from utils import utils
from datetime import datetime

print(utils.date_to_utc_timestamp(datetime(2023, 5, 23, 1, 0)))

1684803600


In [24]:
df.sort_values(by=["time"], ascending=True, inplace=True)
df.head(10)

,time,arb_high,arb_low,kucoin_high,kucoin_low,arb_volume,kucoin_volume,high_diff,high_diff_rate,low_diff,low_diff_rate
0,1684800000,1818.25555,1818.25389,1818.27,1817.80,462.874455,99368.32,-0.01445,0.000008,0.45389,0.000250
1,1684800060,1818.25784,1818.25545,1818.40,1818.17,2996.957906,69844.82,-0.14216,0.000078,0.08545,0.000047
2,1684800180,1818.37415,1818.24957,1818.99,1818.41,32740.669877,184832.72,-0.61585,0.000339,-0.16043,0.000088
3,1684800240,1818.37420,1818.37420,1818.93,1818.52,55.016836,85728.63,-0.55580,0.000306,-0.14580,0.000080
4,1684800300,1818.37403,1818.37399,1819.17,1818.85,32740.751811,49105.83,-0.79597,0.000438,-0.47601,0.000262
5,1684800360,1818.63848,1818.49863,1819.44,1819.13,36735.875500,104732.32,-0.80152,0.000441,-0.63137,0.000347
6,1684800420,1818.64388,1818.63848,1819.45,1819.26,3237.218458,61450.85,-0.80612,0.000443,-0.62152,0.000342
7,1684800480,1818.64111,1818.64111,1819.43,1819.33,214.860556,21242.51,-0.78889,0.000434,-0.68889,0.000379
8,1684800540,1818.64193,1818.64193,1819.43,1819.29,4.116292,13132.65,-0.78807,0.000433,-0.64807,0.000356
9,1684800720,1818.64195,1818.64195,1819.33,1819.16,3443.965077,57162.59,-0.68805,0.000378,-0.51805,0.000285


In [30]:
df[df["time"] >= 1684805340].head(200)

,time,arb_high,arb_low,kucoin_high,kucoin_low,arb_volume,kucoin_volume,high_diff,high_diff_rate,low_diff,low_diff_rate
84,1684805340,1832.09506,1823.41922,1832.58,1823.74,2.627307e+06,6600945.07,-0.48494,0.000265,-0.32078,0.000176
85,1684805400,1839.50176,1831.78591,1841.35,1831.74,2.081068e+06,3721682.26,-1.84824,0.001005,0.04591,0.000025
86,1684805460,1840.27585,1838.37083,1841.61,1839.12,4.872914e+05,1233587.28,-1.33415,0.000725,-0.74917,0.000408
87,1684805520,1839.52977,1838.98115,1840.15,1839.06,1.885188e+05,2182820.27,-0.62023,0.000337,-0.07885,0.000043
88,1684805580,1838.84391,1838.32013,1840.19,1838.28,9.366620e+04,406585.13,-1.34609,0.000732,0.04013,0.000022
...,...,...,...,...,...,...,...,...,...,...,...
279,1684817040,1858.04763,1856.92268,1858.45,1856.87,1.759482e+05,517803.58,-0.40237,0.000217,0.05268,0.000028
280,1684817100,1859.03868,1857.96531,1859.09,1857.85,1.807542e+05,264824.06,-0.05132,0.000028,0.11531,0.000062
281,1684817160,1859.03958,1859.02414,1859.12,1858.31,3.933063e+03,398156.13,-0.08042,0.000043,0.71414,0.000384
282,1684817220,1859.02429,1859.01766,1859.07,1858.36,1.322605e+03,118311.25,-0.04571,0.000025,0.65766,0.000354


In [28]:
kucoin_df[kucoin_df["time"] == 1684805340]

,time,high,low,open,volumefrom,volumeto,close,conversionType,conversionSymbol
89,1684805340,1832.58,1823.74,1824.13,3608.9,6600945.07,1831.74,direct,NaN


In [ ]:
# CEX & DEX trading volume high
